In [ ]:
import pandas as pd

Ouvrire le fichier de pharmakon

In [ ]:
PKON_file = r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\pkon23all-Chmochampv14nov.xlsx"

In [ ]:
pkon_df = pd.read_excel(PKON_file)

Ne récupérer que les colonnes INCHIKEY; CLASSYFIRE_CLASS et CLASSYFIRE_SUBCLASS du fichier de pkon

In [ ]:
nouveau_df = pkon_df[['INCHIKEY', 'CLASSYFIRE_CLASS', 'CLASSYFIRE_SUBCLASS']]

In [ ]:
nouveau_df['INCHIKEY'] = nouveau_df['INCHIKEY'].astype(str)

ne récupérer que les short InChiKey

In [ ]:
nouveau_df['INCHIKEY'] = nouveau_df['INCHIKEY'].apply(lambda x: x.split('-')[0] if x is not None else None)

In [ ]:
nouveau_df.head() # Affiche pour vérification

Ouvrire mon fichier où nous avons l'ensemble des InChiKey de nos bases de données

In [ ]:
MSP_file = r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\FragBank_CLEAN\Nov2023\CSV\ALL_DB_INCHIKEY.xlsx"

In [ ]:
MSP_df = pd.read_excel(MSP_file)

In [ ]:
MSP_df['INCHIKEY'] = MSP_df['INCHIKEY'].astype(str)

N'en récupérer égakement que les short InChiKey

In [ ]:
MSP_df['INCHIKEY'] = MSP_df['INCHIKEY'].apply(lambda x: x.split('-')[0] if x is not None else None)

In [ ]:
MSP_df.head() # Affichage pour vérification

On complète nos InChiKey de nos bases de données avec les classyfire de PKON23

In [ ]:
MSP_df = MSP_df.merge(nouveau_df[['INCHIKEY', 'CLASSYFIRE_CLASS', 'CLASSYFIRE_SUBCLASS']], on='INCHIKEY', how='left')

In [ ]:
MSP_df = MSP_df.drop_duplicates(subset='INCHIKEY', keep='first')

In [ ]:
MSP_df.head() # Affichage pour vérification

On enregistre ce DF là

In [ ]:
MSP_df.to_excel(r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\MSP_inchikeys_classyfire.xlsx", index=False)

In [ ]:
null_df = MSP_df[MSP_df['CLASSYFIRE_CLASS'].isnull()] # NULL

In [ ]:
null_df = null_df.drop_duplicates(subset='INCHIKEY', keep='first')

ici on enregistre le DF des short InChikey qui n'ont aucune information de classification taxonomique ==> c'est celui-ci que l'on utilise pour l'interrogation API de Classyfire

In [ ]:
null_df.to_excel(r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\MSP_inchikeys_classyfire_null.xlsx", index=False)

Juste pour voir ce qu'on à réussi à récupérer de PKON 23

In [ ]:
not_null_df = MSP_df[~MSP_df['CLASSYFIRE_CLASS'].isnull()] # NOT NULL

In [ ]:
not_null_df.to_excel(r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\MSP_inchikeys_classyfire_not_null.xlsx", index=False)